In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as patches
from matplotlib.lines import Line2D
import datetime
import scipy
import os

import VVSAnalyzer
import MBTAnalyzer
import vid2vib_utils
import spectra_utils
import aggregate_utils
import plotting_utils
import accelerometer_utils

## Site Specific Parameters

In [ ]:
# video to process
vid_num = '5486'

# parametrized path
vid_path = f'../TreeSwayData/sidneytrees/videos/SidneyTrees_30s_{vid_num}.MP4'

# dictionary storing datetimes for each video
vid_datetimes = {'5486' : '2020-8-15 12:06',
                 '5731' : '2020-8-20 17:28',
                 '6236' : '2020-8-31 11:50'}

# dictionary storing rois
rois = {'w293' : [420, 520, 395, 445],
        'control460' : [400, 575, 690, 760],
        'region1' : [336, 732, 200, 904]}

# roi to analyze
roi = 'w293'

# directory to store output (optional)
out_prefix = f'./output/manitou_{vid_num}_{roi}'

## Video Processing Parameters

In [ ]:
# VSS video translation kwargs
vid2vib_pa_kwargs = {'reduction':'gray', 'edge':False}

# MBT video translation kwargs
vid2vib_mbt_kwargs = {'reduction':'gray'}
nlevels = 8

# upper and lower bound of candidate frequencies
freqmin = 0.15
freqmax = 0.5

# function and kwargs used to estimate PSD
spectra_func = spectra_utils.get_spectra_periodogram
spectra_func_kwargs = {'window':'boxcar', 'nfft':4096}

# function and kwargs used to find dominant frequency in each spectrum
dom_freq_func = spectra_utils.get_dom_freq_peak_finder
dom_freq_func_kwargs = {}

# masking parameters
percentile=75
masks = [{'name':f'Prominence, {percentile}th percentile',
          'weight':'prominence',
          'kwargs':{'stat_reduc':'percentile', 'percentile':percentile}}, 
        ]

# plotting figure size
figsize=(14,7)

## VVS

In [ ]:
vvs = VVSAnalyzer.VVSAnalyzer(vid_path, rois[roi])

In [ ]:
vib_vvs, fps = vvs.vid2vib(vid2vib_pa_kwargs)

In [ ]:
freq_vvs, pxx_vvs = vvs.compute_spectra(freqmin, freqmax, spectra_func, spectra_func_kwargs)

In [ ]:
agg_df_vvs, pxx_avg_vvs, pxx_avg_peak_idx_vvs, dom_freq, masked, masked_avg_spectrums, masked_avg_spectrums_peak_idxs = vvs.aggregate(dom_freq_func, dom_freq_func_kwargs, masks)

In [ ]:
vvs.report(figsize=figsize)

## VVS Batch Processing

In [ ]:
num_vids = 22

vids = [30 + i for i in range(num_vids)]

vid_path = '../TreeSwayData/sidneytrees/batch_0831/GOPR62{num}.MP4'
vid_dir = '../TreeSwayData/sidneytrees/batch_0831/'

vid_paths = os.listdir(vid_dir)

time_start = '2020-8-31 7:46'
datetimes = [pd.to_datetime(time_start) + pd.to_timedelta(str(30 * i) + 'm' for i in range(num_vids))]

In [ ]:
avg = []
mode = []

for v in sorted(vid_paths):
    path = vid_dir + v
    print(f'Processing video: {path}')
    vvs = VVSAnalyzer.VVSAnalyzer(path, rois[roi])
    vib_vvs, fps = vvs.vid2vib(vid2vib_pa_kwargs)
    freq_vvs, pxx_vvs = vvs.compute_spectra(freqmin, freqmax, spectra_func, spectra_func_kwargs)
    agg_df_vvs, pxx_avg_vvs, pxx_avg_peak_idx_vvs, dom_freq, masked, masked_avg_spectrums, masked_avg_spectrums_peak_idxs = vvs.aggregate(dom_freq_func, dom_freq_func_kwargs, masks)
    
    avg.append(agg_df_vvs['mask0_avg_spectrum_peak'][0])
    mode.append(agg_df_vvs['mask0_mode'][0])

In [ ]:
plt.scatter(datetimes[0], mode[1:])

## MBT

In [ ]:
mbt = MBTAnalyzer.MBTAnalyzer(vid_path, rois[roi], nlevels, out_prefix)

In [ ]:
vib_mbt, _, levels = mbt.vid2vib(vid2vib_mbt_kwargs)

In [ ]:
freq_mbt, pxx_mbt = mbt.compute_spectra(freqmin, freqmax, spectra_func, spectra_func_kwargs)

In [ ]:
agg_df_mbt, pxx_avg_mbt, pxx_avg_peak_idx_mbt = mbt.aggregate(dom_freq_func, dom_freq_func_kwargs, masks)

In [ ]:
mbt.report(figsize=figsize)